In [96]:
import os
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

In [97]:
import torch
torch.cuda.is_available()

True

In [98]:
CLASSES = [
    'almond', 'banana', 'cardamom', 'Cherry', 'chilli', 'clove', 'coconut',
    'Coffee-plant', 'cotton', 'Cucumber', 'Fox_nut(Makhana)', 'gram', 'jowar',
    'jute', 'Lemon', 'maize', 'mustard-oil', 'Olive-tree', 'papaya',
    'Pearl_millet(bajra)', 'pineapple', 'rice', 'soyabean', 'sugarcane',
    'sunflower', 'tea', 'Tobacco-plant', 'tomato', 'vigna-radiati(Mung)', 'wheat'
]


CLASSES_PATH = "C:\\Code\\image-crops\\Cops_DB\\Agricultural-crops"
IMG_SIZE = (128, 128)
TRAIN_IMAGES = 24
VAL_IMAGES = 8         

device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cuda'

In [99]:
def load_and_preprocess_data():
    X_train, y_train = [], []
    X_val, y_val = [], []

    print("Carregando imagens...\n")

    for class_idx, class_name in enumerate(CLASSES):
        class_path = os.path.join(CLASSES_PATH, class_name)

        if not os.path.exists(class_path):
            print(f"Aviso: Pasta {class_name} não encontrada!")
            continue

        image_files = [f for f in os.listdir(class_path)
                       if f.lower().endswith(('.jpg', '.jpeg', '.png'))][:32]

        train_files = image_files[:TRAIN_IMAGES]
        val_files = image_files[TRAIN_IMAGES:32]

        for img_file in train_files:
            try:
                img = Image.open(os.path.join(class_path, img_file)).convert('RGB')
                img = img.resize(IMG_SIZE)
                img_array = np.array(img)/255.0
                img_array = np.transpose(img_array,(2,0,1))
                X_train.append(img_array)
                y_train.append(class_idx)
            except Exception as e:
                print("Erro:", e)

        for img_file in val_files:
            try:
                img = Image.open(os.path.join(class_path, img_file)).convert('RGB')
                img = img.resize(IMG_SIZE)
                img_array = np.array(img)/255.0
                img_array = np.transpose(img_array,(2,0,1))
                X_val.append(img_array)
                y_val.append(class_idx)
            except Exception as e:
                print("Erro:", e)

        print(f"Classe {class_name}: Treino {len(train_files)}, Validação {len(val_files)}")

    X_train = torch.tensor(np.array(X_train), dtype=torch.float32)
    y_train = torch.tensor(np.array(y_train), dtype=torch.long)
    X_val = torch.tensor(np.array(X_val), dtype=torch.float32)
    y_val = torch.tensor(np.array(y_val), dtype=torch.long)

    print("\nDataset final:")
    print("Treino:", X_train.shape, "| Val:", X_val.shape)

    return X_train, y_train, X_val, y_val


In [100]:
X_train, y_train, X_val, y_val = load_and_preprocess_data()

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=16, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=16, shuffle=False)

print("Loaders prontos!")

Carregando imagens...

Classe almond: Treino 21, Validação 0
Classe banana: Treino 24, Validação 7
Classe cardamom: Treino 22, Validação 0
Classe Cherry: Treino 24, Validação 8
Classe chilli: Treino 23, Validação 0
Classe clove: Treino 24, Validação 6
Classe coconut: Treino 24, Validação 1
Classe Coffee-plant: Treino 24, Validação 5
Classe cotton: Treino 24, Validação 8
Classe Cucumber: Treino 24, Validação 7
Classe Fox_nut(Makhana): Treino 23, Validação 0
Classe gram: Treino 24, Validação 1
Classe jowar: Treino 24, Validação 6
Classe jute: Treino 23, Validação 0
Classe Lemon: Treino 24, Validação 4
Classe maize: Treino 24, Validação 7
Classe mustard-oil: Treino 24, Validação 4
Classe Olive-tree: Treino 24, Validação 6
Classe papaya: Treino 23, Validação 0
Classe Pearl_millet(bajra): Treino 24, Validação 8
Classe pineapple: Treino 24, Validação 1
Classe rice: Treino 24, Validação 5
Classe soyabean: Treino 24, Validação 6
Classe sugarcane: Treino 24, Validação 1
Classe sunflower: Treino

In [101]:
def create_model(num_classes):
    return nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=3, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),  # 128 -> 64

        nn.Conv2d(32, 64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),  # 64 -> 32

        nn.Conv2d(64, 128, kernel_size=3, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),  # 32 -> 16

        nn.Conv2d(128, 256, kernel_size=3, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),  # 16 -> 8

        nn.Flatten(),

        nn.Linear(256 * 8 * 8, 512),  # ALTERADO!
        nn.ReLU(),
        nn.Dropout(0.5),

        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Dropout(0.5),

        nn.Linear(256, num_classes)
    )


In [102]:
num_classes = len(CLASSES)

model = create_model(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

model

Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU()
  (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): ReLU()
  (15): MaxPool2d(kernel_size=2, stri

In [103]:
def accuracy(preds, labels):
    _, preds_max = torch.max(preds, 1)
    return (preds_max == labels).float().mean().item()


In [104]:
EPOCHS = 100

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0
    running_acc = 0

    for images, labels in tqdm(train_loader, desc=f"Treinando Epoch {epoch+1}/{EPOCHS}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_acc += accuracy(outputs, labels)

    # print(f"\nEpoch {epoch+1}: Loss={running_loss/len(train_loader):.4f}, "
    #       f"Acc={running_acc/len(train_loader):.4f}")


Treinando Epoch 100/100: 100%|██████████| 45/45 [00:01<00:00, 30.01it/s]


In [105]:
model.eval()
val_loss = 0
val_acc = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        loss = criterion(outputs, labels)
        val_loss += loss.item()
        val_acc += accuracy(outputs, labels)

print(f"\nValidação -> Loss={val_loss/len(val_loader):.4f}, Accuracy={val_acc/len(val_loader):.4f}")



Validação -> Loss=2.7240, Accuracy=0.2268


In [106]:
# def predict_single_image(path):
#     img = Image.open(path).convert("RGB")
#     img = img.resize(IMG_SIZE)
#     img = np.array(img)/255.0
#     img = np.transpose(img, (2,0,1))
#     img = torch.tensor(img, dtype=torch.float32).unsqueeze(0).to(device)

#     model.eval()
#     with torch.no_grad():
#         output = model(img)
#         pred = torch.argmax(output)

#     return CLASSES[pred.item()]

# predict_single_image("exemplo.jpg")
